# ***creation de graphiques d'analyses des résultats post-traitements***

In [ ]:
%load_ext autoreload
%autoreload 2
import geopandas as gp
import pandas as pd
import numpy as np
import altair as alt
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
from MMM import import_fichiers_mmm
import Connexion_Transfert as ct
import random
from shapely.ops import unary_union
from collections import Counter

import Affectation_pt_comptage as ap
import Estim_trafic as et

# 1.Analyse des sources de données

> ## 1.1 fichiers sources

In [ ]:
resGrp1=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\FV_affectation_complete_grp1.shp')
res=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\FV_affectation_complete.shp')
mmm_simple,cle_mmm_rhv=import_fichiers_mmm(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\MMM\MMM_Maj_EDA2017\simplifie\MMM_simple_2019.shp',
                                            r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\appariementV0.shp')
MMMsimple2019NonNull=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\MMM\MMM_Maj_EDA2017\simplifie\MMM_simple_2019_nonNull.shp')
pour_calcul=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\pour_calcul.shp')

rdpt=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\rdpoint_ext.shp')

In [ ]:
cbs=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\CBS2017\Filaire_global_metropole_DENSIFIER.shp')
otv2017=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\OTV_2017\lin2017_BdTref_2017.shp')
traf_voies_1234=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_1234_cpt.shp')

In [ ]:
ppvCptFv=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\ppv_cpt_FV.shp')

In [ ]:
bati_final=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\bati.shp')

In [6]:
len(bati_source)

637692

In [ ]:
bati_source=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\bati\PopulationMajic_033_2016\BdTopoAvecPop_2016_D33.shp')

> ## 1.2. Comptages  
Analyse des points de comptages : calcul de la longueur des tronçons RHV supportant un type_cpt "permanent" ou "ponctuel", par classe de RHV, par rapport à la longueur totale des tronçons de cette classe

In [5]:
#preparation des données
ppvCptFv=ppvCptFv.loc[~ppvCptFv.cat_rhv.isna()].copy()
ppvCptFv['long']=ppvCptFv.geometry.length/1000
ppvCptFv['cat_rhv_simple']=ppvCptFv.cat_rhv.apply(lambda x : x[-1])
ppvCptFv['grp_indic']=ppvCptFv.apply(lambda x : 'comptage' if not pd.isnull(x['ident_cpt']) else 'autre', axis=1)

In [6]:
baseComptageParCatRhv=alt.Chart(ppvCptFv.groupby(['cat_rhv_simple','grp_indic']).long.sum().reset_index()).mark_bar().encode(
    x=alt.X('cat_rhv_simple', title='Catégorie RHV simplifiée'),
    color=alt.Color('grp_indic',title='Source des trafics'))
chartComptageParCatRhv=baseComptageParCatRhv.encode(y=alt.Y('long', title='longueur (km)')) | baseComptageParCatRhv.encode(
    y=alt.Y('long', title='longueur (km)', stack='normalize', axis=alt.Axis(format='%')))

In [7]:
chartComptageParCatRhv

alt.HConcatChart(...)

> ## 1.3. MMM  
Comparaison des données de trafics des voies du MMM avec celle issues des points de comptages, par catégorie du RHV. Il faut pour ça enlever les ronds points du RHV

In [13]:
#preparation des données
resGrp1=resGrp1.loc[~resGrp1.cat_rhv.isna()].copy()
resGrp1['long']=resGrp1.geometry.length/1000
resGrp1['cat_rhv_simple']=resGrp1.cat_rhv.apply(lambda x : x[-1])
resGrp1['grp_indic']=resGrp1.apply(lambda x : 'comptage' if x['type_cpt'] in ('permanent','ponctuel') else 'autre', axis=1)

In [14]:
#rond_points
listIdentRdpt=gp.sjoin(res,rdpt,op='within').ident.tolist()

In [15]:
#preparer les donées pour compraraison : enlver la Rocade et autres 2*2 voies de la comparaison et calcul des écarts puis on ne garde que le valeur max du MMM pour chaque ligne si doublons
CompMMMCpt=cle_mmm_rhv[['NO','ident']].merge(pour_calcul.loc[(pour_calcul.nom_sect.isin(['intra-bld', 'extra-Rocade_rd', 'extra-Rocade-Sud_rg',
       'extra-Rocade-NO_rg', 'intra-Rocade-Sud_rg', 'Bastide','intra-Rocade-NO_rg', 'intra-Rocade_rd', 'rocade bretelle','Bld RG'])) & 
        (~pour_calcul.ident.isin(listIdentRdpt)) & (pour_calcul.type_cpt.isin(('permanent','ponctuel')))][['ident','cat_rhv_si','tmjo_2_sen','nom_sect', 'numero']], 
                                             on='ident').merge(mmm_simple[['NO','tmja_tv']])
CompMMMCpt.drop_duplicates(['tmjo_2_sen','tmja_tv'], inplace=True)
CompMMMCpt['ecart']=CompMMMCpt.tmjo_2_sen-CompMMMCpt.tmja_tv
CompMMMCpt['ecart_cpt']=CompMMMCpt.tmjo_2_sen/CompMMMCpt.tmja_tv
CompMMMCpt=CompMMMCpt.loc[CompMMMCpt.groupby(['tmjo_2_sen']).tmja_tv.transform(max)==CompMMMCpt.tmja_tv].copy()

In [16]:
MMMsimple2019NonNull.loc[MMMsimple2019NonNull.calc_lg!=0].geometry.length.sum()/1000

1344.370559441787

In [17]:
resGrp1.loc[resGrp1.cat_rhv_simple!='4'].geometry.length.sum()/1000

1214.8206447216899

In [18]:
#calcul des graphs
chartDensiteProbabiliteEcart=alt.Chart(CompMMMCpt.loc[(CompMMMCpt.ecart_cpt<12)],width=325, height=220).transform_density(
    'ecart_cpt',groupby=['cat_rhv_si'],    as_=['ecart_cpt', 'density'],
).mark_area().encode(
    x=alt.X("ecart_cpt:Q",title= 'rapport tmjo mesuré / tmja MMM'),
    y=alt.Y('density:Q', title='densité de probablité'),
    facet=alt.Facet('cat_rhv_si:N', columns=1, title=None))

chartQuantileEcart=alt.vconcat(*[
(alt.Chart(CompMMMCpt.loc[(CompMMMCpt.ecart_cpt<12) & (CompMMMCpt.cat_rhv_si==str(i))], title=f'catégorie RHV : {str(i)}').transform_quantile(
    'ecart_cpt', step=0.01,groupby=['cat_rhv_si']
).mark_line().encode(
    x=alt.X('prob:Q', axis=alt.Axis(tickMinStep=0.05)) ,
    y=alt.Y('value:Q', scale=alt.Scale(domain=[0,12]))) + 
alt.Chart(pd.DataFrame({'x':[e/10 for e in range(0,11)], 'y':[0.6]*11, 'y2':[1.3]*11})).mark_area(opacity=0.3, color='green').encode(
    x=alt.X('x', title = 'quantile'), 
    y=alt.Y('y', title= 'rapport tmjo mesuré / tmja MMM'), y2='y2') 
).properties(width=300, height=200) for i in range(1,4)])

chartExampleVariation=alt.Chart(CompMMMCpt.loc[(0.3<CompMMMCpt.ecart_cpt) | (2<CompMMMCpt.ecart_cpt) & (CompMMMCpt.ecart_cpt<12)],width=300, height=450).mark_circle().encode(
    x=alt.X('tmjo_2_sen', title='tmjo mesuré'), 
    y=alt.Y('tmja_tv', title='tmja MMM'),
    tooltip=['numero','ident'],
    color='numero',
    text='numero',
    facet=alt.Facet('cat_rhv_si:N', columns=3, title=None)).resolve_scale(
    color='independent', y='independent', x='independent')

In [19]:
alt.hconcat(chartQuantileEcart,chartDensiteProbabiliteEcart)

alt.HConcatChart(...)

In [20]:
chartExampleVariation

alt.Chart(...)

>## 1.4 fichiers CBS

In [21]:
#calcul tmja_tv
cbs['tmjatv']=cbs.apply(lambda x : ((float(x.MT.replace(',','.'))+float(x.PT.replace(',','.')))*12) + 
                        ((float(x.ME.replace(',','.'))+float(x.PE.replace(',','.')))*4) + 
                        ((float(x.MT.replace(',','.'))+float(x.PT.replace(',','.')))*8), axis=1)

#dans le fichier CBS trouver les valeurs les plus représenté pour supposer que ce sont des valeurs par défaut
Counter(cbs.tmjatv.tolist()).most_common(10)
#de ça on en déduit que les 6 valeusr les plus représentées sont des estimations, on va mettre un attribut de marquage
listTmjaAttributaire=[e[0] for e in Counter(cbs.tmjatv.tolist()).most_common(6)]
cbs['typeTmja']=cbs['tmjatv'].apply(lambda x : 'Arbitraire' if x in listTmjaAttributaire else 'Mesure')
cbs['long_km']=cbs.geometry.length/1000
cbs['nature']='CBS'

In [22]:
#preparation des données
res=res.loc[~res.cat_rhv.isna()].copy()
res['long']=res.geometry.length/1000
res['cat_rhv_simple']=res.cat_rhv.apply(lambda x : x[-1])
res['grp_indic']=res.apply(lambda x : 'comptage' if x['type_cpt'] in ('permanent','ponctuel') else 'autre', axis=1)

In [23]:
#pareil pour otv2017, Chart de comparaison
otv2017['long_km']=otv2017.geometry.length/1000
otv2017['typeTmja']=otv2017.coment_cpt.apply(lambda x : 'Mesure' if x=='linearisation' else 'Arbitraire')
otv2017['nature']='OTV'

alt.Chart(pd.concat([otv2017[['long_km','typeTmja','nature']], cbs[['long_km','typeTmja','nature']]]).groupby(['typeTmja','nature']).sum().reset_index()).mark_bar().encode(
    y=alt.Y('sum(long_km):Q', stack='normalize', axis=alt.Axis(format='%', title='Longueur (km)')),
    x=alt.X('nature:N', title = 'Référentiel'),
    color=alt.Color('typeTmja', title='Type de TMJA')
)

alt.Chart(...)

In [19]:
#transfert de données par buffer puis concatenation
trafic_comptage=res.loc[(res['grp_indic']=='comptage') & (~res.ident.isin(listIdentRdpt))].merge(traf_voies_1234[['ident','idtronc','rgraph_dbl','id_cpt_exp']], on='ident')
trafic_comptage['buffer']=trafic_comptage.geometry.buffer(3)
trafic_comptage=trafic_comptage.set_geometry('buffer')
idtroncBuff=trafic_comptage[['buffer','idtronc']].dissolve(by='idtronc').reset_index()
#idtroncBuff.to_file(r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\buffCompCbsTraCpt.shp')
#regrouper les buffers par id_tronc en gardant le type de voie le plus représenté
test=trafic_comptage.groupby('idtronc').agg({'buffer' : lambda x : unary_union(list(x)),
                                       'rgraph_dbl' : lambda x : Counter(tuple(x)).most_common(1)[0][0],
                                       'id_cpt_exp' : lambda x : Counter(tuple(x)).most_common(1)[0][0], 
                                            'tmjo_2_sen' : lambda x : Counter(tuple(x)).most_common(1)[0][0],
                                            'cat_rhv_simple' : lambda x : Counter(tuple(x)).most_common(1)[0][0]}).reset_index().set_geometry('buffer')
test.crs='epsg:2154'
#joindre spatialement les données 
jointureComplete=gp.sjoin(cbs[['geometry','CAT_DIG','NAME','NOM','tmjatv']],test.to_crs('epsg:3945'), op='within').merge(traf_voies_1234[['idtronc','tmjo_2_sen']].drop_duplicates(), on='idtronc')#on regroupe les valuers de tmja différente par idtronc pour les cbs
groupTmjatv=jointureComplete.groupby('idtronc').agg({'tmjatv' : lambda x : tuple(set(x))}).reset_index().merge(test)
#²pour comparer on va se limiter aux troncons pour lesuqles ona 1 seule valeur de trafic ou alors 2 valuer de trafics de cahque cote sur des lignes a sens uniques
CompCbsTraf=groupTmjatv.loc[groupTmjatv.apply(lambda x : len(x.tmjatv) < 2 or ((len(x.tmjatv) == 2) and (x.rgraph_dbl==0) and ',' in x.id_cpt_exp), axis=1)].copy()
#calcul du trafic final cbs pour comparaison
CompCbsTraf['traf_final']=CompCbsTraf.tmjatv.apply(lambda x : sum(x))
CompCbsTraf['ecart_cpt']=CompCbsTraf.tmjo_2_sen/CompCbsTraf.traf_final

In [20]:
CompCbsTraf['typeTmjaCbs']=CompCbsTraf.traf_final.apply(lambda x : 'Attributaire' if x in listTmjaAttributaire else 'mesure')

In [21]:
chartCompCbsTraf=alt.Chart(CompCbsTraf[['tmjo_2_sen','traf_final','typeTmjaCbs']]).mark_point().encode(x='tmjo_2_sen',y='traf_final', color='typeTmjaCbs')
chartCompCbsTraf + chartCompCbsTraf.transform_regression('tmjo_2_sen', 'traf_final').mark_line().interactive()

alt.LayerChart(...)

In [22]:
#calcul des graphs
chartDensiteProbabiliteEcart=alt.Chart(CompCbsTraf.loc[(CompCbsTraf.ecart_cpt<12)][['ecart_cpt','cat_rhv_simple']],width=325, height=220).transform_density(
    'ecart_cpt',groupby=['cat_rhv_simple'],as_=['ecart_cpt', 'density'],
).mark_area().encode(
    x=alt.X("ecart_cpt:Q",title= 'rapport tmjo mesuré / tmja CBS'),
    y=alt.Y('density:Q', title='densité de probablité'),
    facet=alt.Facet('cat_rhv_simple:N', columns=1, title=None))

In [23]:
chartDensiteProbabiliteEcart

alt.Chart(...)

In [24]:
chartQuantileEcart=alt.vconcat(*[
(alt.Chart(CompCbsTraf.loc[(CompCbsTraf.ecart_cpt<12) & (CompCbsTraf.cat_rhv_simple==str(i))][['ecart_cpt','cat_rhv_simple']], title=f'catégorie RHV : {str(i)}').transform_quantile(
    'ecart_cpt', step=0.01,groupby=['cat_rhv_simple']
).mark_line().encode(
    x=alt.X('prob:Q', axis=alt.Axis(tickMinStep=0.05)) ,
    y=alt.Y('value:Q', scale=alt.Scale(domain=[0,12]))) + 
alt.Chart(pd.DataFrame({'x':[e/10 for e in range(0,11)], 'y':[0.6]*11, 'y2':[1.3]*11})).mark_area(opacity=0.3, color='green').encode(
    x=alt.X('x', title = 'quantile'), 
    y=alt.Y('y', title= 'rapport tmjo mesuré / tmja CBS'), y2='y2') 
).properties(width=300, height=200) for i in range(1,5)])

chartExampleVariation=alt.Chart(CompCbsTraf.loc[(0.3<CompCbsTraf.ecart_cpt) | (2<CompCbsTraf.ecart_cpt) & (CompCbsTraf.ecart_cpt<12)][['tmjo_2_sen','traf_final',
    'cat_rhv_simple','typeTmjaCbs']],width=300, height=450).mark_circle().encode(
    x=alt.X('tmjo_2_sen', title='tmjo mesuré'), 
    y=alt.Y('traf_final', title='tmja CBS'),
    color='typeTmjaCbs',
    facet=alt.Facet('cat_rhv_simple:N', columns=3, title=None)).resolve_scale(
    color='independent', y='independent', x='independent')

In [25]:
chartExampleVariation

alt.Chart(...)

# 2. Affectation des points de comptages
Ici l'objectif est simplement de rappeler que les graphs présentés au chapitre décrivant les point sde compatgesprennent en compte l'affectation des trafics à des idtroncons, et que si ce n'était pas le cas, on trouverait bien moins de longueur

In [24]:
#preparation des données
res=res.loc[~res.cat_rhv.isna()].copy()
res['long']=res.geometry.length/1000
res['cat_rhv_simple']=res.cat_rhv.apply(lambda x : x[-1])
res['grp_indic']=res.apply(lambda x : 'comptage' if x['type_cpt'] in ('permanent','ponctuel') else 'autre', axis=1)

In [25]:
#analyse du pourcentage de longueur ayant fait l'objet de comptage
baseComptageParCatRhvAffecte=alt.Chart(res.groupby(['cat_rhv_simple','grp_indic']).long.sum().reset_index()).mark_bar().encode(
    x=alt.X('cat_rhv_simple', title='Catégorie RHV simplifiée'),
    color=alt.Color('grp_indic',title='Source des trafics'))
chartComptageParCatRhvAffecte=baseComptageParCatRhvAffecte.encode(y=alt.Y('long', title='longueur (km)')) | baseComptageParCatRhvAffecte.encode(
    y=alt.Y('long', title='longueur (km)', stack='normalize', axis=alt.Axis(format='%')))

In [26]:
chartComptageParCatRhv.properties(title='Répartition des trafics routiers sans affectation')|chartComptageParCatRhvAffecte.properties(title='Répartition des trafics routiers avec affectation')

alt.HConcatChart(...)

# 3. Estimation des voies de catégories RHV 1 à 3


In [27]:
#rapport entre les troncon renseigné manuellement et les autres, pour les voies de catégories 1 à 3 
len(et.dico_corection_calcul_3_voies)/traf_voies_1234.loc[~traf_voies_1234.cat_rhv.isin(('4','64'))].idtronc.nunique()

0.07359125315391085

In [37]:
#analyse du pourcentage de longueur selon la source de calcul l'objet de comptage
baseComptageParCatRhv123Estim=alt.Chart(res.loc[(~res.cat_rhv.isin(('4','64'))) & (res.type_cpt!='estimation_catRhv4')].groupby(['cat_rhv_simple','type_cpt']).long.sum().reset_index()).mark_bar().encode(
    x=alt.X('cat_rhv_simple', title='Catégorie RHV simplifiée'),
    color=alt.Color('type_cpt',title='Source des trafics'))
chartComptageParCatRhv123Estim=baseComptageParCatRhv123Estim.encode(y=alt.Y('long', title='longueur (km)')) | baseComptageParCatRhv123Estim.encode(
    y=alt.Y('long', title='longueur (km)', stack='normalize', axis=alt.Axis(format='%')))

In [38]:
chartComptageParCatRhv123Estim

alt.HConcatChart(...)

# 4. Estimation des voies de catégories RHV 4


In [34]:
len(res.loc[(res.type_cpt=='estimation_catRhv4') & (~res.cat_rhv.isin(('4','64')))])

55

In [13]:
#filtre des bâtiments
bati_final.loc[bati_final.ID=='BatiFictif']
bati_final.loc[bati_final.PopT2016<1]
alt.Chart(pd.DataFrame({'x':['Total', 'Population < 1', 'Bati Fictif'], 'y':[len(bati_final), len(bati_final.loc[bati_final.PopT2016<1]), 
        len(bati_final.loc[bati_final.ID=='BatiFictif'])]})).mark_bar().encode(x='x',y='y')

alt.Chart(...)

In [ ]:
#filtre des établiseemenet
